In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer    
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
dataset = load_boston()
X_full, y_full = dataset.data, dataset.target
n_samples = X_full.shape[0]         #506
n_features = X_full.shape[1]        #13

In [ ]:
rng = np.random.RandomState(0)
missing_rate = 0.5
n_missing_samples = int(np.floor(n_samples * n_features * missing_rate))    #3289

#randint（下限，上限，n）指在下限和上限之间取出n个整数
missing_features = rng.randint(0, n_features, n_missing_samples)    
missing_samples = rng.randint(0,n_samples,n_missing_samples)

X_missing = X_full.copy()
y_missing = y_full.copy()

## 制造缺失值
X_missing[missing_samples, missing_features] = np.nan
X_missing = pd.DataFrame(X_missing)

## 把特征进行排序

In [ ]:
X_missing_reg = X_missing.copy()
sortindex = np.argsort(X_missing_reg.isnull().sum(axis=0)).values   
## 有缺失值的特征按缺失值个数从大到小排列
sortindex

## 逐个填充

In [ ]:
for i in sortindex:
    #构建我们的新特征矩阵（没有被选中去填充的特征 + 原始的标签）和新标签（被选中去填充的特征）
    df = X_missing_reg
    fillc = df.iloc[:, i]   #新标签
    df = pd.concat([df.iloc[:,df.columns != i], pd.DataFrame(y_full)], axis=1)#新特征矩阵
    
    #在新特征矩阵中，对含有缺失值的列，进行0的填补
    df_0 =SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0).fit_transform(df)
                        
    #找出我们的训练集和测试集
    Ytrain = fillc[fillc.notnull()] # Ytrain是被选中要填充的特征中（现在是我们的标签），存在的那些值：非空值
    Ytest = fillc[fillc.isnull()]   # Ytest 是被选中要填充的特征中（现在是我们的标签），不存在的那些值：空值。注意我们需要的不是Ytest的值，需要的是Ytest所带的索引
    Xtrain = df_0[Ytrain.index, :]   # 在新特征矩阵上，被选出来的要填充的特征的非空值所对应的记录
    Xtest = df_0[Ytest.index, :]     # 在新特征矩阵上，被选出来的要填充的特征的空值所对应的记录
    
    #用随机森林回归来填补缺失值
    rfc = RandomForestRegressor(n_estimators = 100) #实例化
    rfc = rfc.fit(Xtrain, Ytrain)                   #导入训练集进行训练
    Ypredict = rfc.predict(Xtest)                   #用predict接口将Xtest导入，得到我们的预测结果（回归结果），就是我们要用来填补空值的这些值
    
    #将填补好的特征返回到我们的原始的特征矩阵中
    X_missing_reg.loc[X_missing_reg.iloc[:, i].isnull(), i] = Ypredict

#检验是否有空值
X_missing_reg.isnull().sum()